# How to use `cugraph-nx` to accelerate graph analysis while using networkX 

Using the cugraph plugin for NetworkX requires Python 3.9, 3.10, or 3.11, in addition to the latest development version of NetworkX. Follow the instructions from `INSTALL.rst` on the NetworkX repository for instructions on installing the development version of `networkx`

First, check which version of networkx you have installed. The cugraph plugin is only on the development version of NetworkX, `networkx-3.2rc0-dev0`. If the below command does not show this version, you will need to follow the instructions from `INSTALL.rst` on the NetworkX repository.

In [3]:
!pip show networkx

Name: networkx
Version: 2.8.8
Summary: Python package for creating and manipulating graphs and networks
Home-page: https://networkx.org/
Author: Aric Hagberg
Author-email: hagberg@lanl.gov
License: 
Location: /opt/conda/envs/rapids/lib/python3.10/site-packages
Requires: 
Required-by: cfn-lint, mapclassify, osmnx, python-louvain, scikit-image


## FOR REPRODUCIBILITY / DEV WORK / TEMP

The following cells will
- uninstall the current version of networkx 
- clone the networkx repository one directory below the current working directory
- upgrade networkx to be the current development version with `pip install`

You may need to restart the kernel to have the development version of networkX be visible.

In [5]:
!pip uninstall --yes networkx

In [12]:
# Skip if current networkx repository already exists
!(cd ..; \
git clone https://github.com/networkx/networkx.git)

Cloning into 'networkx'...
remote: Enumerating objects: 65975, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 65975 (delta 3), reused 4 (delta 2), pack-reused 65942
Receiving objects: 100% (65975/65975), 91.34 MiB | 17.01 MiB/s, done.
Resolving deltas: 100% (43970/43970), done.


In [6]:
!(cd ../networkx; \
pip install --upgrade --target=/opt/conda/envs/rapids/lib/python3.10/site-packages -e .[default])
# below: old pip install command,
# pip install --target=/opt/conda/envs/rapids/lib/python3.10/site-packages -e .[default])

Obtaining file:///dylanc/week4/networkx
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 22.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 18.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 33.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 41.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 48.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━

In [10]:
!pip show networkx

Name: networkx
Version: 3.2rc0.dev0
Summary: Python package for creating and manipulating graphs and networks
Home-page: 
Author: 
Author-email: Aric Hagberg <hagberg@lanl.gov>
License: 
Location: /opt/conda/envs/rapids/lib/python3.10/site-packages
Editable project location: /dylanc/week4/networkx
Requires: 
Required-by: cfn-lint, mapclassify, osmnx, python-louvain, scikit-image


In [1]:
import networkx
networkx, networkx.__version__

(<module 'networkx' from '/dylanc/week4/networkx/networkx/__init__.py'>,
 '3.2rc0.dev0')

## Using cugraph backend

In [21]:
import networkx as nx
import pandas as pd

In [14]:
nx, nx.__version__, pd.__version__

('3.2rc0.dev0', '2.0.3')

In [15]:
df = pd.read_csv("datasets/dolphins.csv", header=None, names=['src', 'dst', 'wgt'], sep=' ')
Gnx = nx.from_pandas_edgelist(df, source='src', target='dst', create_using=nx.Graph())

In [16]:
%%timeit
bc = nx.betweenness_centrality(Gnx)

14 ms ± 400 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Build cugraph / not required?

In [23]:
!pip show cugraph

Name: cugraph
Version: 23.6.0
Summary: cuGraph - RAPIDS GPU Graph Analytics
Home-page: 
Author: NVIDIA Corporation
Author-email: 
License: Apache 2.0
Location: /opt/conda/envs/rapids/lib/python3.10/site-packages
Requires: cudf, cupy-cuda11x, dask, dask-cuda, dask-cudf, distributed, numba, pylibcugraph, raft-dask, rmm, ucx-py
Required-by: cugraph-dgl, cugraph-pyg, cugraph-service-server


In [24]:
!(./build.sh libcugraph pylibcugraph cugraph cugraph-nx --skip_cpp_tests)

Building for the architecture of the GPU in the system...
-- Auto detection of gpu-archs: 70
-- Building for GPU_ARCHS = 70-real
-- CPM: adding package libcudacxx@1.9.1 (branch/1.9.1)
-- rapids-cmake [libcudacxx]: applied diff install_rules.diff to fix issue: 'libcudacxx 1.X installs incorrect files [https://github.com/NVIDIA/libcudacxx/pull/428]'
-- rapids-cmake [libcudacxx]: applied diff reroot_support.diff to fix issue: 'Support conda-forge usage of CMake rerooting [https://github.com/NVIDIA/libcudacxx/pull/490]'
-- CPM: using local package cuco@0.0.1
-- CPM: using local package raft@23.10.0
-- CPM: using local package cugraph-ops@23.10.0
-- Configuring done (51.0s)
CMake Warning at CMakeLists.txt:304 (add_library):
  Cannot generate a safe runtime search path for target cugraph because files
  in some directories may conflict with libraries in implicit directories:

    runtime library [libcugraph-ops++.so] in /opt/conda/envs/all_cuda-120_arch-x86_64/lib may be hidden by files in:


## Customization

Depending on your graph needs, `networkx` supports the ability to choose your dispatcher. A dispatchable networkX algorithm will use the chosen dispatcher to convert networkx Graph to the backend equivalent, dispatch the work, then convert back to the required nx form. More information here: https://networkx.org/documentation/stable/reference/classes/index.html#backends

In [17]:
from networkx.utils import backends

In [18]:
# Show how to select the name of the backend you want networkx to dispatch to
backends.plugin_name = "cugraph"
orig_dispatch = backends._dispatch
#test_dispatch = backends.test_override_dispatch
#dispatch_decorator = orig_dispatch
#backends.plugins

In [20]:
backends.plugin_name

'cugraph'

## What algorithms are supported?

|        Algorithm            |  Type         | Implemented |   Notes
| ----------------------------|---------------|-------------| ---------
| Betweeneess Centrality      | Centrality    |      X      | cuGraph 23.10 nightly         
| Edge Betweenness Centrality | Centrality    |      X      | cuGraph 23.10 nightly
| Louvain                     | Community     |             | cuGraph 23.10 nightly (expected)

## Future plans

We want to get your feedback! As this project develops, we want you to voice your opinions on which algorithms we should prioritize.